# T5 = sport, lifeandstyle

크롤러

In [ ]:
import torch
torch.__version__

'2.2.1+cu121'

In [ ]:
import requests
import json
import datetime as dt
import pandas as pd

In [ ]:
# import pytz
# import datetime
# # 미국 서부 표준시 (PST) 타임존 설정
# pst_timezone = pytz.timezone('US/Pacific')

# # 현재 시각을 PST 타임존으로 변환
# pst_now = now.astimezone(pst_timezone)

# # PST 기준 날짜 문자열 형식으로 변환
# pst_now_date = pst_now.strftime("%Y-%m-%d")

# print(f"현재 PST 날짜: {pst_now_date}")

In [ ]:
# now = dt.datetime.now()
# now_date = now.strftime("%Y-%m-%d")
# now_date

## 테이블 만들기

## API 가능한지 함수


## 크롤링 함수

In [ ]:
# my_api_key = "6de253c2-9d98-4b21-9750-e4e9b63c8d28"
# section = "lifeandstyle"
# date = "2024-04-10"

def guardian_search(my_api_key, section, date):
  url = "https://content.guardianapis.com/" + section
  final_url = url + "?from-date=" + date + "&to-date=" + date +"&show-fields=bodyText&page-size=200&api-key=" + my_api_key
  response = requests.get(final_url)
  news_data_json = response.content.decode("utf-8")

  # results 부분만 추출
  news_data_crawl = json.loads(news_data_json)
  news_data_crawl_results = news_data_crawl["response"]["results"]

  # results 에서 원하는 부분만 리스트로 추출
  result_list = []
  for search_crawl_context in news_data_crawl_results:
    result_dict = {}
    # result_dict['sectionId'] = search_crawl_context["sectionId"]
    result_dict['article_public_Date'] = search_crawl_context["webPublicationDate"]
    result_dict['title'] = search_crawl_context["webTitle"]
    result_dict['content_url'] = search_crawl_context["webUrl"]
    result_dict['content'] = search_crawl_context["fields"]["bodyText"]
    result_dict["category_id"] = 3
    result_list.append(result_dict)

  result_list

  # df = pd.DataFrame(result_list)

  return result_list

In [ ]:
my_api_key = "6de253c2-9d98-4b21-9750-e4e9b63c8d28"
section = "business"
date = "2024-04-09"

news_list = guardian_search(my_api_key, section, date)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

## 딥러닝 모델 요약해서 리턴

In [ ]:
!pip install transformers

위 리스트에서 bodyText 부분만 뽑아서 리스트로 만들고 그걸 하나씩 s

In [ ]:
from transformers import AutoModelWithLMHead, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-summarize-news")
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-summarize-news")

def summarize(text, max_length=150):
  input_ids = tokenizer.encode(text, return_tensors="pt", add_special_tokens=True)

  generated_ids = model.generate(input_ids=input_ids, num_beams=2, max_length=max_length,  repetition_penalty=2.5, length_penalty=1.0, early_stopping=True)

  preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]

  return preds[0]



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1595: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [ ]:
#summarize('Guardian production editor Nikki Marshall was in the Bondi Westfield when the attack began. She recounts how she escaped the centre: I was on the fourth floor of the David Jones department store at Bondi Junction Westfield at 3.35pm when another shopper came up behind me and said: “There’s been a stabbing, it’s happening – a shooting, stabbing – we have to go.”  I could see in her face that she had seen something, and it was real. I think she was clutching her daughter’s hand. I headed towards the rear of the store, looking for a fire escape. I found a cupboard with a fire extinguisher in it. It was tiny and dark and I was afraid to close the door fully, so decided it didn’t give me much protection and I kept moving. Around a corner I found a fire escape with half a dozen shoppers standing outside it. A few were saying, should we leave? I realised they’d only heard shouting and weren’t sure what was happening.', 150)

In [ ]:
summary_list = []

for item in news_list:
    body_text = item['content']
    summary = summarize(body_text, 150)
    summary_list.append(summary)

Token indices sequence length is longer than the specified maximum sequence length for this model (535 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
print(summary_list)

["2.7 million people sought help from a lender, debt adviser or other financial support charity in the year up to January, according to Financial Conduct Authority. However, 47% of those who had fallen behind on their bills said they had avoided talking to their lender about their finances. The regulator has reminded consumers in financial difficulty that lenders are obliged to listen to customers' concerns. Notably, 7.4 million people in the UK struggled to pay a bill or credit repayment in January.", 'I have been borrowing more than ever to cover normal expenses, including medical bills. Special education teacher Chris from Colorado owes over $47,000 in loans due to unaffordable healthcare bills and essential home repairs over the past three years. He also took out a $7,000 loan to help repay the first loan. Interest rates on credit cards are currently at 5.5%, which is 25 years ago. Americans are considering leaving the US, according to reports. The cost of living in the US has gone

## 요약한 거 json 파일에 열 추가

In [ ]:
def add_abstracts(news_list, summary_list):

  for i, item in enumerate(news_list):
    # 요약본 추출
    summary = summary_list[i]

    # abstract 열 추가 및 요약본 저장
    item['abstract'] = summary

# abstract 열 추가
add_abstracts(news_list, summary_list)

# 결과 확인 (result_list에는 각 문서의 요약본이 'abstract' 열에 담겨있음)
print(news_list)

[{'article_public_Date': '2024-04-09T23:01:31Z', 'title': 'Paying bills a struggle for 7.4 million UK consumers, regulator finds  ', 'content_url': 'https://www.theguardian.com/business/2024/apr/10/paying-bills-a-struggle-for-74-million-uk-consumers-regulator-finds', 'content': 'More than 7.4 million people in the UK struggled to pay a bill or a credit repayment in January, according to a financial regulator. The figure is less than last year but is still significantly higher than before the cost of living crisis began. According to the Financial Conduct Authority (FCA), which tracks the number of households in financial difficulties, 5.8 million people reported that they were struggling to pay a large bill in February 2020. The data, which gives an insight into the UK’s personal finances, showed that by January 2023, after the Russian invasion of Ukraine and the subsequent cost of living crisis, the number of people in financial difficulties almost doubled to 10.9 million. The regulat

In [ ]:
print(news_list)

[{'article_public_Date': '2024-04-09T23:01:31Z', 'title': 'Paying bills a struggle for 7.4 million UK consumers, regulator finds  ', 'content_url': 'https://www.theguardian.com/business/2024/apr/10/paying-bills-a-struggle-for-74-million-uk-consumers-regulator-finds', 'content': 'More than 7.4 million people in the UK struggled to pay a bill or a credit repayment in January, according to a financial regulator. The figure is less than last year but is still significantly higher than before the cost of living crisis began. According to the Financial Conduct Authority (FCA), which tracks the number of households in financial difficulties, 5.8 million people reported that they were struggling to pay a large bill in February 2020. The data, which gives an insight into the UK’s personal finances, showed that by January 2023, after the Russian invasion of Ukraine and the subsequent cost of living crisis, the number of people in financial difficulties almost doubled to 10.9 million. The regulat

In [ ]:
import json


with open("business_04_09.json", "w") as f:
    json.dump(news_list, f, indent=4)

이 데이터를 sql로 적재함

-이러면 웨어하우스는 없이 바로 마트로 만드는 건지??

## SQL에 저장

In [ ]:
# business.info()